In [3]:
#Evaluating German WSD

In [1]:
import os, yaml, re
import together

In [2]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)
    
os.environ['TOGETHER_AI_API'] = credentials['TOGETHER_AI_API']
together.api_key = os.environ["TOGETHER_AI_API"]

In [3]:
from together import Together

In [4]:
def complete_model(USER_MESSAGE):
    output = together.Complete.create(
                                    prompt= USER_MESSAGE,
                                    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
                                    max_tokens=512,
                                    temperature=0,
                                    top_p=0.7,
                                    top_k=50,
                                    repetition_penalty=1,
                                    stop=["<|eot_id|>"]
                                    
                                    
                                    
                                    )
    text = output['output']['choices'][0]['text']
    return text

In [1]:
#!python -m spacy download fr_core_news_sm


In [5]:
import spacy
nlp = spacy.load("fr_core_news_sm")


wordnet_dict={}
with open ("inventory.fr.txt","r") as file:
    for i in file:
        parts=i.split("\t")
        element=i.split("\t")[0]
        word,pos=element.split("#")[0],element.split("#")[1]
        value = "\t".join(parts[1:]).strip()
        doc = nlp(word)
        lemmatized_words = [token.lemma_ for token in doc]


        #print(lemmatized_words[0]+"\t"+pos+"\t"+value)

        # meanings = get_babelnet_sense_meaning(sense_id)
        # print(meanings[0])

        #save all the data in a dictionary keeping the word as key and the senses as values in a list
        wordnet_dict[word]=value

In [ ]:
import requests

def get_babelnet_sense_meaning(sense_id):
    # Define the BabelNet API endpoint
    url = f'https://babelnet.io/v6/getSynset'
    
    # Define the parameters for the API request
    params = {
        'id': sense_id,
        'key': "#"
    }
    
    # Make the API request
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Get the English glosses (definitions)
        #print(data)
        glosses = data.get('glosses', [])
        english_glosses = [gloss['gloss'] for gloss in glosses if gloss['language'] == 'EN']

        lemmas = []
        for sense in data.get('senses', []):
            properties = sense.get('properties', {})
            lemma_info = properties.get('lemma', {})
            lemma = lemma_info.get('lemma')
            if lemma:
                lemmas.append(lemma)


        senses = data.get('senses', [])
        senseKey = None

        if senses:
            sense_properties = senses[0].get('properties', {})
            senseKey = sense_properties.get('senseKey', None)


        #print(lemmas)

        return english_glosses[0],lemmas[0],senseKey
    else:
        print(f"Error: Unable to fetch data (status code: {response.status_code})")
        return None


In [8]:

# glosses,lemma,sensekey_wn=get_babelnet_sense_meaning("bn:00003651n")

In [9]:
# print(glosses)
# print(lemma)
# print(sensekey_wn)

A sloping gallery with seats for spectators (as in an operating room or theater)
amphitheater
amphitheater%1:06:01::


In [7]:
# Read data from file
wordnet = []
with open('wordnet.txt', 'r',encoding="utf8") as file:
    for element in file:
        items=element.strip("\n").split("	")
        wordnet.append(items)
    

In [8]:
def extract_fourth_element(lists_of_lists, key):
    for sublist in lists_of_lists:
        if sublist[0] == key:
            return sublist[3]
    return None

In [11]:
# result = extract_fourth_element(wordnet, sensekey_wn)
# print(result)

In [13]:
# import random
# from collections import defaultdict

# def get_balanced_examples(input_list):
#     # Parse the input list into a dictionary with senseid as key
#     senseid_dict = defaultdict(list)
    
#     for item in input_list:
        
#         example, senseid = item.rsplit(',', 1)
#         senseid_dict[senseid.strip()].append(item.strip())  # Store the full item (example + senseid)
    
#     #Find the minimum number of examples for any senseid
#     min_examples = min(len(examples) for examples in senseid_dict.values())
    
#     #Randomly sample min_examples from each senseid
#     balanced_list = []
#     for senseid, examples in senseid_dict.items():
#         balanced_list.extend(random.sample(examples, min_examples))
    
#     return balanced_list

In [9]:
import random
from collections import defaultdict

def get_balanced_examples(input_list):
    # Parse the input list into a dictionary with senseid as key
    senseid_dict = defaultdict(list)
    
    for item in input_list:
        
        example, senseid = item.rsplit(',', 1)
        senseid_dict[senseid.strip()].append(item.strip())  # Store the full item (example + senseid)
    
    #Find the minimum number of examples for any senseid
    avg_examples = min(len(examples) for examples in senseid_dict.values()) + max(len(examples) for examples in senseid_dict.values())
    avg_examples=int(avg_examples/2)

    
    #Randomly sample min_examples from each senseid
    balanced_list = []
    for senseid, examples in senseid_dict.items():
        balanced_list.extend(random.sample(examples, avg_examples))
    
    return balanced_list

In [10]:
#Funtion to counts the number of tokens : input and output tokens
from nltk.tokenize import word_tokenize

def count_tokens(text):
    # Tokenize the input text
    tokens = word_tokenize(text)
    
    # Count the tokens in the input
    token_count = len(tokens)
    
    return token_count

In [11]:
import json
def load_dictionary_from_file(filename):
    try:
        with open(filename, 'r') as file:
            data_dict = json.load(file)
        return data_dict
    except FileNotFoundError:
        # Handle the case where the file doesn't exist
        return None
    except json.JSONDecodeError:
        # Handle the case where the file contains invalid JSON
        return None

In [12]:
loaded_dict = load_dictionary_from_file("my_dictionary.json")

In [13]:
#identifying the WSD word from the data 
def wsdword(text):
    match = re.search(r'<WSD>(.*?)<WSD>', text)
    if match:
        word_inside_wsd = match.group(1)
        return word_inside_wsd

In [14]:
def sense_Tag_Return_pipeline(sentence,postag,wordwsd,wordnet):
    
    #retrieving the base form of the sentence
    doc = nlp(wordwsd)
    lemmatized_words = [token.lemma_ for token in doc]
    lemmatized_word =lemmatized_words[0].lower()
    #print(lemmatized_word)
    #retrieving the data form the KB (retriver)
    try:
        examples=loaded_dict[wordwsd][postag]
        #Fewshot bias Removal
        examples=get_balanced_examples(examples)


        #print(examples)
    except:
        examples="No Examples"
        #print(examples)

    #retrieving the meanings from Bablenet
    sense_definition=""
    list_senseids=wordnet_dict[lemmatized_word].split("\t")
    #print(list_senseids)
    for i in list_senseids:
        glosses,lemma,sensekey_wn=get_babelnet_sense_meaning(i)
        result = extract_fourth_element(wordnet, sensekey_wn)
        senseinfo="senseid: "+i+", gloss :"+glosses+ ", lemma in English :"+lemma +", Synonyms of the word if any :"+result+"\n"
        sense_definition+=senseinfo

   
    #print(sense_definition)
    
        

    count=0 #variable to count the tokens 
    
    #prompt=f"Examine the sentence. {instance_meaning}.Return most suitable sense id associated with from below. it contain sense id and it's definition {meanings}. utilize the below examples also to finalize the answer {examples}"
    prompt = f'''You are going to identify the corresponding sense tag of an ambiguous word in french sentences. Use multiple reasoning strategies to increase confidence in your answer.
1. The word "{wordwsd}" has different meanings. Below are possible meanings. Comprehend the sense tags and meanings. English lemma and synonyms are provided. {sense_definition}
2. You can learn more on the usage of each word and the its sense through the examples below. Each sentence is followed by its corresponding sense id. "{examples}"
3. Now carefully examine the German sentence below. The ambiguous word is enclosed within <WSD>."{sentence}"
4. Analyze the sentence using the following techniques and identify the meaning of the ambiguous word.
   Focus on keywords in the sentence surrounding the ambiguous word. 
   Think about the overall topic and intent of the sentence. Decide on the sense of the word that makes the most logical sense within the context. 
5. Based on the identified meaning, try to find the most appropriate senseID from the above sense tag list. You are given definition of each sense tag too.
6. Return only the finalized senseID as a JSON object. Do not add extra details and explanation. Only senseID as a JSON is expected.
   > '''
    #print(prompt)
    count+=count_tokens(prompt)
    output=complete_model(prompt)
    count+=count_tokens(output)
    return output, count    


In [ ]:
import json
import csv

#evaluating the results
no_of_instances=0
no_of_correct_instances=0

csv_file_path = 'test/sem13_fr.csv'
totalTokens=0
# Open the CSV file
with open(csv_file_path, mode='r', newline='' , encoding="utf-8") as file:
    # Create a CSV reader object
    csv_reader = csv.DictReader(file)
    
    # Iterate over each row in the CSV file
    
    for row in csv_reader:
        # Access the columns by their header names
        sentence = row['sentence']
        fileid = row['fileid']
        senseid = row['senseid'].strip()        
        
        try:
            postag=senseid[-1]  #postag 
            wordfromtext=wsdword(sentence)
            #print(wordfromtext+" "+senseid+" "+postag+" "+sentence)            
            output,token_count=sense_Tag_Return_pipeline(sentence,postag,wordfromtext.lower(),wordnet)
            #sense_Tag_Return_pipeline(sentence,postag,wordfromtext)
            totalTokens+=token_count
            print(output.replace("\n", ""))
            no_of_instances+=1
            if senseid in output:
                no_of_correct_instances+=1
            
            
        except:
            print()
            
        
    

file.close()
print("No of Total instance :",no_of_instances)
print("No of correct instance :",no_of_correct_instances)
print(totalTokens)

In [ ]:
import json
import csv

#evaluating the results
no_of_instances=0
no_of_correct_instances=0

csv_file_path = 'test/sem13_fr.csv'
totalTokens=0
# Open the CSV file
with open(csv_file_path, mode='r', newline='' , encoding="utf-8") as file:
    # Create a CSV reader object
    csv_reader = csv.DictReader(file)
    
    # Iterate over each row in the CSV file
    
    for row in csv_reader:
        # Access the columns by their header names
        sentence = row['sentence']
        fileid = row['fileid']
        senseid = row['senseid'].strip()        
        
        try:
            postag=senseid[-1]  #postag 
            wordfromtext=wsdword(sentence)
            #print(wordfromtext+" "+senseid+" "+postag+" "+sentence)            
            output,token_count=sense_Tag_Return_pipeline(sentence,postag,wordfromtext.lower(),wordnet)
            #sense_Tag_Return_pipeline(sentence,postag,wordfromtext)
            totalTokens+=token_count
            print(output.replace("\n", ""))
            no_of_instances+=1
            if senseid in output:
                no_of_correct_instances+=1
            
            
        except:
            print()
            
        
    

file.close()
print("No of Total instance :",no_of_instances)
print("No of correct instance :",no_of_correct_instances)
print(totalTokens)